In [ ]:
#!/usr/bin/env python
# coding: utf-8
import sys
sys.path.append('..')
import umbrella_mesh
import elastic_rods
import linkage_vis
from umbrella_mesh import UmbrellaEnergyType
from bending_validation import suppress_stdout as so
from visualization_helper import *

import pipeline_helper, importlib, design_optimization_analysis
with so(): importlib.reload(pipeline_helper)
with so(): importlib.reload(design_optimization_analysis)

from pipeline_helper import UmbrellaOptimizationCallback, allEnergies, allGradientNorms, allDesignObjectives, allDesignGradientNorms, set_joint_vector_field, show_center_joint_normal, show_joint_normal

from design_optimization_analysis import DesignOptimizationAnalysis

import umbrella_optimization
import umbrella_optimization_finite_diff
from umbrella_optimization import OptEnergyType

import numpy as np
import numpy.linalg as la

import pickle, gzip

from configuration import *

from datetime import datetime

import os

from load_jsondata import read_data, write_deformed_config
import mesh
import importlib, pipeline_helper

from equilibrium_solve_analysis import EquilibriumSolveAnalysis
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-8
OPTS.verbose = 1
OPTS.beta = 1e-6
OPTS.niter = 300
OPTS.verboseNonPosDef = False

import time


import umbrella_optimization

import py_newton_optimizer
opt_opts = py_newton_optimizer.NewtonOptimizerOptions()
opt_opts.gradTol = 1e-8
opt_opts.verbose = 10
opt_opts.beta = 1e-6
opt_opts.niter = 600
opt_opts.verboseNonPosDef = False


import pickle 
import gzip

import compute_vibrational_modes

from configuration import *

In [ ]:
# ### Initialization
name = 'tigridia'
input_path = '../../data/{}.json.gz'.format(name)
io, input_data, target_mesh, curr_um, thickness, target_height_multiplier = parse_input(input_path)
# curr_um = pickle.load(gzip.open('../../Optimized_model/tigridia/Copy of tigridia_optimized_rest_state_2022_01_23_16_15_target_height_factor_5.0.pkl.gz'))
curr_um = pickle.load(gzip.open('../../python/optimization_experiments/2022_01_25_14_46_tigridia/tigridia_input_equilibrium_2022_01_25_14_46_target_height_factor_5.0.pkl.gz', 'r'))

In [ ]:
# #### Pin Rigid Motion
# 
# 
use_pin = False

driver = curr_um.centralJoint()
jdo = curr_um.dofOffsetForJoint(driver)
fixedVars = (list(range(jdo, jdo + 6)) if use_pin else []) + curr_um.rigidJointAngleDoFIndices()

In [ ]:
rod_colors = get_color_field(curr_um, input_data)

view = linkage_vis.LinkageViewerWithSurface(curr_um, target_mesh, width=1024, height=512)
set_surface_view_options(view, color = 'green', surface_color = 'gray', umbrella_transparent = False, surface_transparent = True)
view.averagedMaterialFrames = True
view.showScalarField(rod_colors)
view.show()

In [ ]:
# np.save('{}_cam_view.npy'.format(name), view.getCameraParams())

In [ ]:
# view.setCameraParams(tuple([tuple(pt) for pt in np.load('{}_cam_view.npy'.format(name))]))

In [ ]:
import time
time_stamp = time.strftime("%Y_%m_%d_%H_%M")
import os
output_folder = '{}_{}'.format(time_stamp, name)
if not os.path.exists(output_folder):
    os.makedirs(output_folder)  

### Offscreen render

In [ ]:
import OffscreenRenderer

In [ ]:
width = 3840
height = 2160

In [ ]:
render = view.offscreenRenderer(width, height)

In [ ]:
def render_callback(prob, i):
    view.showScalarField(rod_colors)
    geometry = view.getVisualizationGeometry()
    render.updateMeshData(geometry[0], geometry[2], rod_colors.colors())
    vw.writeFrame()

In [ ]:
def eqm_callback(prob, i):
    view.showScalarField(rod_colors)

In [ ]:
def optimization_render():
    geometry = view.getVisualizationGeometry()
    render.updateMeshData(geometry[0], geometry[2], rod_colors.colors())
    vw.writeFrame()

### Undeployment

In [ ]:
# use_pin = False

# driver = curr_um.centralJoint()
# jdo = curr_um.dofOffsetForJoint(driver)
# fixedVars = (list(range(jdo, jdo + 6)) if use_pin else []) + curr_um.rigidJointAngleDoFIndices()

# configure_umbrella_undeployment_step_one(curr_um, thickness, target_height_multiplier)

# OPTS.niter = 10

# results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)
# results.success
# configure_umbrella_undeployment_step_two(curr_um)

# curr_um.attractionWeight = 0

# OPTS.niter = 100

# results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)
# results.success

# pickle.dump(curr_um, gzip.open('{}_rest_state.pkl.gz'.format(name), 'w'))

### Deployment

In [ ]:
# use_pin = False

# driver = curr_um.centralJoint()
# jdo = curr_um.dofOffsetForJoint(driver)
# fixedVars = (list(range(jdo, jdo + 6)) if use_pin else []) + curr_um.rigidJointAngleDoFIndices()



# configure_umbrella_pre_deployment(curr_um, thickness, target_height_multiplier)

# break_input_angle_symmetry(curr_um)
# results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)

### TSF Optimization

In [ ]:
import pipeline_helper, importlib
with so(): importlib.reload(pipeline_helper)

In [ ]:
# ### Initialize Design Optimization
configure_umbrella_optimization(curr_um, bdryMultiplier = 1)

import py_newton_optimizer
opt_opts = py_newton_optimizer.NewtonOptimizerOptions()
opt_opts.gradTol = 1e-8
opt_opts.verbose = 10
opt_opts.beta = 1e-6
opt_opts.niter = 600
opt_opts.verboseNonPosDef = False

results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = opt_opts, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)

opt_opts.niter = 50

results.success

# Run target surface fitting first

optimizer = umbrella_optimization.UmbrellaOptimization(curr_um, opt_opts, 2.5, -1, False, fixedVars)

In [ ]:
vw = OffscreenRenderer.video_writer.MeshRendererVideoWriter('{}_tsf_optimization.mp4'.format(name), render, outWidth = 1920, outHeight = 1080)

In [ ]:
optimizer.beta = 1 * 1e6
optimizer.gamma = 1
optimizer.eta = 0
optimizer.zeta = 0# 1e1
optimizer.iota = 0

rest_height_optimizer = umbrella_optimization.UmbrellaRestHeightsOptimization(optimizer)
single_rest_height_optimizer = umbrella_optimization.UmbrellaSingleRestHeightOptimization(rest_height_optimizer)

rest_height_optimizer.newPt(rest_height_optimizer.params())

doptays = DesignOptimizationAnalysis(rest_height_optimizer)

pipeline_helper.prev_time_stamp = time.time()

uo = rest_height_optimizer.get_parent_opt()

uo.equilibriumOptimizer.options.verbose = 1
#uo.equilibriumOptimizer.options.verboseWorkingSet = True
uo.equilibriumOptimizer.options.gradTol = 1e-10
# Hold the closest points fixed in the target-attraction term of the equilibrium solve:
# this seems to make the design optimization much more robust.
uo.setHoldClosestPointsFixed(True, False)
tfview = pipeline_helper.TargetFittingVisualization(curr_um, uo.target_surface_fitter, view)
cb = pipeline_helper.UmbrellaOptimizationCallback(rest_height_optimizer, view, True, False, 1, rod_colors, doptays.record, tfview=tfview, osrender = optimization_render)

In [ ]:
algorithm = umbrella_optimization.OptAlgorithm.NEWTON_CG
#algorithm = umbrella_optimization.OptAlgorithm.BFGS
arm_length_lower_bound = input_data["plate_edge_length"] / 30 * 32

In [ ]:
solverStatus = umbrella_optimization.optimize(rest_height_optimizer, algorithm, 200, 0.005, 1e-5, cb, arm_length_lower_bound)

In [ ]:
rest_height_optimizer.reset_joint_target_with_closest_points()
curr_um.scaleInputPosWeights(0.1, bdryMultiplier = 2.0)

solverStatus = umbrella_optimization.optimize(rest_height_optimizer, algorithm, 100, 0.005, 1e-5, cb, arm_length_lower_bound)

In [ ]:
rest_height_optimizer.beta = 1 * 1e7
rest_height_optimizer.reset_joint_target_with_closest_points()
curr_um.scaleInputPosWeights(0.1, bdryMultiplier = 1.0)

solverStatus = umbrella_optimization.optimize(rest_height_optimizer, algorithm, 100, 0.005, 1e-5, cb, arm_length_lower_bound)

In [ ]:
vw.finish()

In [ ]:
np.save('{}/{}_tsf_arm_lengths.npy'.format(output_folder, name), doptays.armLengths)

In [ ]:
# np.save('{}_tsf_cam_view.npy'.format(name), view.getCameraParams())

### Force Optimization

In [ ]:
vw = OffscreenRenderer.video_writer.MeshRendererVideoWriter('{}_force_optimization.mp4'.format(name), render, outWidth = 1920, outHeight = 1080)

In [ ]:

optimizer.beta = 1 * 1e6
optimizer.gamma = 1
optimizer.eta = 0
optimizer.zeta = 0# 1e1
optimizer.iota = 1e10

import force_analysis
cfm = force_analysis.UmbrellaForceMagnitudes(curr_um)

normalActivationThreshold = min(np.percentile(cfm[:, 0], 30), 0)

optimizer.objective.terms[-1].term.normalActivationThreshold = normalActivationThreshold

optimizer.objective.terms[-1].term.normalWeight = 1
optimizer.objective.terms[-1].term.tangentialWeight = 0
optimizer.objective.terms[-1].term.torqueWeight = 0

rest_height_optimizer = umbrella_optimization.UmbrellaRestHeightsOptimization(optimizer)
single_rest_height_optimizer = umbrella_optimization.UmbrellaSingleRestHeightOptimization(rest_height_optimizer)

rest_height_optimizer.newPt(rest_height_optimizer.params())

doptays = DesignOptimizationAnalysis(rest_height_optimizer)

import time
pipeline_helper.prev_time_stamp = time.time()

uo = rest_height_optimizer.get_parent_opt()

uo.equilibriumOptimizer.options.verbose = 1
#uo.equilibriumOptimizer.options.verboseWorkingSet = True
uo.equilibriumOptimizer.options.gradTol = 1e-10
# Hold the closest points fixed in the target-attraction term of the equilibrium solve:
# this seems to make the design optimization much more robust.
uo.setHoldClosestPointsFixed(True, False)
cb = pipeline_helper.UmbrellaOptimizationCallback(rest_height_optimizer, view, True, False, 1, rod_colors, doptays.record, tfview=tfview, osrender = optimization_render)
algorithm = umbrella_optimization.OptAlgorithm.NEWTON_CG
#algorithm = umbrella_optimization.OptAlgorithm.BFGS
solverStatus = umbrella_optimization.optimize(rest_height_optimizer, algorithm, 300, 0.005, 1e-5, cb, arm_length_lower_bound)


In [ ]:
np.save('{}/{}_force_arm_lengths.npy'.format(output_folder, name), doptays.armLengths)

In [ ]:
vw.finish()

### Render Optimized rest state

In [ ]:
# ### Initialization
name = 'tigridia'
input_path = '../../data/{}.json.gz'.format(name)
io, input_data, target_mesh, rest_um, thickness, target_height_multiplier = parse_input(input_path, handleBoundary=True)

In [ ]:
# #### Pin Rigid Motion
# 
# 
use_pin = True

driver = rest_um.centralJoint()
jdo = rest_um.dofOffsetForJoint(driver)
fixedVars = (list(range(jdo, jdo + 6)) if use_pin else []) + rest_um.rigidJointAngleDoFIndices()

In [ ]:
rod_colors = get_rest_state_color_field(rest_um, input_data)

rv = linkage_vis.LinkageViewer(rest_um, width=1024, height=512)
rv.averagedMaterialFrames = True
rv.update(scalarField=rod_colors)
rv.show()

In [ ]:
tsf_opt_arm_lengths = np.load('{}/{}_tsf_arm_lengths.npy'.format('2022_01_26_16_25_tigridia', name))
# force_opt_arm_lengths = np.load('{}/{}_force_arm_lengths.npy'.format(output_folder, name))

In [ ]:
rest_render = rv.offscreenRenderer(width, height)

In [ ]:
vw = OffscreenRenderer.video_writer.MeshRendererVideoWriter('{}_tsf_opt_rest_state.mp4'.format(name), rest_render, outWidth = 1920, outHeight = 1080)

In [ ]:
def rest_eqm_callback(prob, i):
    rv.update(scalarField=rod_colors)

In [ ]:
OPTS.niter = 500

In [ ]:
for armlengths in tsf_opt_arm_lengths:
    rest_um.setPerArmRestLength(armlengths)
    with so(): results = umbrella_mesh.compute_equilibrium(rest_um, callback = rest_eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)
    rv.update(scalarField=rod_colors)
    print(results.success)
    geometry = rv.getVisualizationGeometry()
    rest_render.updateMeshData(geometry[0], geometry[2], rod_colors.colors())
    vw.writeFrame()

In [ ]:
vw.finish()

### Render tsf true equilibrium state

In [ ]:
# ### Initialization
name = 'tigridia'
input_path = '../../data/{}.json.gz'.format(name)
io, input_data, target_mesh, deployed_um, thickness, target_height_multiplier = parse_input(input_path, handleBoundary=True)
deployed_um = pickle.load(gzip.open('../../python/optimization_experiments/2022_01_25_14_46_tigridia/tigridia_tsf_equilibrium_2022_01_25_14_46_target_height_factor_5.0.pkl.gz', 'r'))

In [ ]:
# #### Pin Rigid Motion
# 
# 
use_pin = True

driver = deployed_um.centralJoint()
jdo = deployed_um.dofOffsetForJoint(driver)
fixedVars = (list(range(jdo, jdo + 6)) if use_pin else []) + deployed_um.rigidJointAngleDoFIndices()

In [ ]:
rod_colors = get_color_field(deployed_um, input_data)

dv = linkage_vis.LinkageViewerWithSurface(deployed_um, target_mesh, width=1024, height=512)
dv.averagedMaterialFrames = True
set_surface_view_options(dv, color = 'green', surface_color = 'gray', umbrella_transparent = False, surface_transparent = True)
dv.showScalarField(rod_colors)
dv.show()

In [ ]:
tsf_opt_arm_lengths = np.load('{}/{}_tsf_arm_lengths.npy'.format('2022_01_26_16_25_tigridia', name))
# force_opt_arm_lengths = np.load('{}/{}_force_arm_lengths.npy'.format(output_folder, name))

In [ ]:
dv.setCameraParams(tuple([tuple(pt) for pt in np.load('{}_tsf_cam_view.npy'.format(name))]))

In [ ]:
deploy_render = dv.offscreenRenderer(width, height)

In [ ]:
vw = OffscreenRenderer.video_writer.MeshRendererVideoWriter('{}_tsf_opt_deployed_state.mp4'.format(name), deploy_render, outWidth = 1920, outHeight = 1080)

In [ ]:
def deployed_eqm_callback(prob, i):
    dv.showScalarField(rod_colors)

In [ ]:
OPTS.niter = 500

In [ ]:
configure_umbrella_pre_deployment(deployed_um, thickness, target_height_multiplier)
curr_um.attractionWeight = 0

In [ ]:
for armlengths in tsf_opt_arm_lengths:
    deployed_um.setPerArmRestLength(armlengths)
    with so(): results = umbrella_mesh.compute_equilibrium(deployed_um, callback = deployed_eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)
    dv.showScalarField(rod_colors)

    print(results.success)
    geometry = dv.getVisualizationGeometry()
    deploy_render.updateMeshData(geometry[0], geometry[2], rod_colors.colors())
    vw.writeFrame()

In [ ]:
vw.finish()

#### Write force optimization rest state

In [ ]:
vw = OffscreenRenderer.video_writer.MeshRendererVideoWriter('{}_force_opt_rest_state.mp4'.format(name), rest_render, outWidth = 1920, outHeight = 1080)

In [ ]:
def rest_eqm_callback(prob, i):
    rv.update(scalarField=rod_colors)

In [ ]:
OPTS.niter = 500

In [ ]:
for armlengths in force_opt_arm_lengths:
    rest_um.setPerArmRestLength(armlengths)
    with so(): results = umbrella_mesh.compute_equilibrium(rest_um, callback = rest_eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)
    rv.update(scalarField=rod_colors)
    print(results.success)
    geometry = rv.getVisualizationGeometry()
    rest_render.updateMeshData(geometry[0], geometry[2], rod_colors.colors())
    vw.writeFrame()

In [ ]:
vw.finish()